In [ ]:
import tensorflow as tf
import os

In [ ]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
tf.config.list_physical_devices('GPU') 


In [ ]:
import cv2
import imghdr

In [ ]:
data_dir='data'

In [ ]:
image_exts=['jpeg','png','jpg','bmp']

In [ ]:
for image_class in os.listdir(data_dir): 
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e: 
            print('Issue with image {}'.format(image_path))
            # os.remove(image_path)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
data=tf.keras.utils.image_dataset_from_directory('data')

In [ ]:
data = data.map(lambda x,y: (x/255, y))

In [ ]:
scaled_iterator=data.as_numpy_iterator()

In [ ]:
batch=scaled_iterator.next()
#class 1 =sad
#class 0=happy

In [ ]:
# Create a figure with 4 side-by-side subplots
fig, ax = plt.subplots(ncols=4, figsize=(20, 20))

# Initialize a counter variable
idx = 0

# Loop through the first 4 images in the batch
for img in batch[0][:4]:
    # Display each image in its respective subplot
    ax[idx].imshow(img)
    # Set the title of each subplot to the corresponding label
    ax[idx].title.set_text(batch[1][idx])
    # Increment the counter
    idx += 1

In [ ]:
train_set=int(len(data)*.7)# batches 
cv_set=int(len(data)*.2)
test_set=int(len(data)*.1)+1

In [ ]:
train = data.take(train_set)
val = data.skip(train_set).take(cv_set)
test = data.skip(train_set+cv_set).take(test_set)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [ ]:
model=Sequential()

In [ ]:
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
logdir='logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [ ]:
for batch in test.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
print(pre.result(), re.result(), acc.result())

In [ ]:
img = cv2.imread('8iAb9k4aT.jpg')
plt.imshow(img)
plt.show()

In [ ]:
resize_img = tf.image.resize(img, (256,256))
plt.imshow(resize_img.numpy().astype(int))
plt.show()

In [ ]:
yhat = model.predict(np.expand_dims(resize_img/255, 0))

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model.save(os.path.join('models','HappySadModel.h5'))

In [ ]:
best_model = load_model(os.path.join('models','HappySadModel.h5'))

In [ ]:
ynewhat = best_model.predict(np.expand_dims(resize_img/255, 0))

In [ ]:
if yhat > 0.5: 
    print(f'Predicted class is Sad')
else:
    print(f'Predicted class is Happy')